# OOP in python
Python is a class-oriented language, which means that classes and objects are the main building blocks.

**Basic principles**
* Class = template for creating objects
* Object = a specific instance of a class
* self = reference to the current instance (equivalent to this in other languages)
* Everything is public - no private or protected, but the _name convention means "internal"
* Inheritance - classes can inherit from other classes (including multiple classes)
* Override - methods can be overridden in a descendant
* All classes automatically inherit from the base class object

## \_\_init\_\_ – class initializer (constructor)

* Special method that is automatically called when the object is created.
* It is used to set the initial state of the object, i.e. to assign values to its attributes.
* The name must not be changed - it must always be \_\_init\_\_.
* The first parameter is self, which is a reference to the instance currently being created.
* If \_\_\_init\_\_ is not in the class, Python will create the object with the default state and will not allow parameters to be passed at initialization.

In [ ]:
from math import sqrt
class Point:
    """ Point in 2d """
    
    def __init__(self, x, y):
        # constructor
        self.x = x
        self.y = y
        self._z = 3      # internal attribute, but it's just a convention, it's easy to get there
        self.__w = 4     # private attribute, not to be used even in descendants, it is more difficult to get there via __dict__
        
    def distance (self, other):
        # method
        return sqrt((other.x - self.x) **2 + (other.y - self.y)**2)

Creating an instance of points

In [ ]:
a = Point (1, 2)
b = Point (4, 5)
print (a.distance(b))   # a is inserted after self automatically

# Inheritance
* Inheritance allows you to create a new class (called child/subclass) that takes over the properties and methods of another class (called parent/superclass).
* It is used to reuse code and to organize classes hierarchically.
* Python also supports multiple inheritance.

In [ ]:
class A:
    def foo(self):
        print ("A.foo()")
        
class B(A):
    def foo(self):
        A.foo(self)          # calling a method from the parent class. Alternatively, you can use super.foo(self)
        print ("B.foo()")

class C(A):
    def foo(self):
        print ("C.foo()")
        
class D(B, C):
    def bar(self):
        print ("D.bar()")        

In [ ]:
b=B()
b.foo()

The inheritance hierarchy can be written using \_\_base\_\_

In [ ]:
D.__bases__

Python uses MRO (Method Resolution Order) - the order in which it looks for methods in multiple inheritance. It is implemented as a linearized list of inheritance hierarchy classes.

A method is called from the class that is found first.

In [ ]:
d=D()
print (d.foo())
print (D.__mro__)

## Instance and class variables
Python allows two main levels of variables in a class, instance and class

* **Instance variables** belong to a specific object (instance). 
    * Each object has its own copy of these variables.
    * They are defined inside the __init__ method using self.

* **Class variables** belong to the class itself, they are shared by all instances.
    * They are defined directly in the class outside of __init__.
    * Access from instance: self.name (if the instance does not have the same attribute)
    * Access from class: Class.name

In [ ]:
class E:
    x = 1           # class variable
    def __init__(self, y):
        self.y=y    # instance variable

In [ ]:
e1 = E(5)
e2 = E(3)
print (E.x, e1.x, e1.y)
print (E.x, e2.x, e1.y)

In [ ]:
e1.x=2
print (E.x, e1.x, e1.y)
print (E.x, e2.x, e1.y)

In [ ]:
E.x=2
print (E.x, e1.x, e1.y)
print (E.x, e2.x, e1.y)

## Static and class methods
There are special categories of methods in Python besides instance methods.

* **Instance method** is a standard class method.
    * The first parameter is always self, which is a reference to a specific instance.
    * They can access both instance and class variables.

* **Class method** (@classmethod)
    * The first parameter is cls, which refers to the class, not the instance.
    * They can only access class variables and other class methods.
    * Decorator: @classmethod    

* **Static method** (@staticmethod)
    * Does not have an automatic self or cls parameter.
    * Works like a regular function, but is part of a class.
    * It does not have access to instance or class variables, but logically belongs to the class.
    * Decorator: @staticmethod 
   

In [ ]:
class Car:
    number_of_wheel = 4  # class variable

    def __init__(self, color):
        self.color = color  # instance variable

    def info(self):
        print(f"Car has {self.number_of_wheel} wheels and color is {self.color}")

    @classmethod
    def info_wheels(cls):
        print(f"All cars has {cls.number_of_wheel} wheels")        

    @staticmethod
    def avg_speed(distance, time):
        return distance / time        

a = Car("red")
a.info()  
Car.info_wheels()
print(Car.avg_speed(120, 2))

## Creating instances

* \_\_new\_\_ - the method creates the instance itself.
   * Called before \_\_\_init\_\_
   * The first parameter is cls, a reference to the class that will be instantiated.
   * Typically used only in advanced patterns (e.g. Singleton).
   * Returns a new instance of the object (return super().\_\_\new\_(cls)).
* \_\_\_init\_\_ - initialization method

In [ ]:
class G:
    def __new__ (cls, x):
        print ('G.__new__()')
        return object.__new__(cls)
    
    def __init__(self, x):
        print ('G.__init__()')
        self.x = x

In [ ]:
g=G(1)

## Storage in memory
In Python, all instance variables of an object are stored in an internal dictionary.

* Each object has its own dictionary accessible via __dict__.
* Key = attribute name, value = reference to the object (data value).
* Very convenient from a programming point of view
* But dictionaries take more memory
* Each variable access needs a hash table lookup 

In [ ]:
e1.__dict__

In [ ]:
e1.z="abc"

In [ ]:
e1.__dict__

## Optimization
If we want to save memory for classes with a large number of instances, we can use __slots__:
* An instance cannot have attributes other than those defined in slots.
* In memory implemented in arrays, access via indexes
* Less volume intensive.
* We lose dynamism

In [ ]:
class H:
    __slots__ = ('x', 'y')
    def __init__ (self, x, y):
        self.x = x
        self.y = y

In [ ]:
h=H(1, 4)

In [ ]:
# ends in an error
h.z=3

## Special methods
In Python, we have several ways to control access to object attributes and free resources.

Python recommends direct access to attributes (object.attribute), but sometimes we need read or write control. The @property decorator and the complementary @<attribute>.setter are used for this purpose.

In [ ]:
class Person:
    def __init__(self, name, age):
        self._name = name      # underscore = convention "private" attribute
        self._age = age

    # getter
    @property
    def age(self):
        return self._age

    # setter
    @age.setter
    def age(self, value):
        if value < 0:
            raise ValueError("Age cannot be negative")
        self._age = value

o = Person("Eva", 25)
print(o.age)
o.age = 30
print(o.age)
o.age = -5

The special method \_\_\_del\_\_ is called when an object is deleted and released by the garbage collector.

It is used to terminate resources (files, sockets, connections, etc.).

In [ ]:
class File:
    def __init__(self, name):
        self.name = name
        self.f = open(name, "w")

    def __del__(self):
        print(f"Closing file {self.name}")
        self.f.close()

s = File("test.txt")
del s 

# Exercise 1
Create an animal class with instance variables:
    * name (str)
    * age (int)

Add an info() method that prints: Name: \<name>, Age: \<age>

In [ ]:
# code

In [ ]:
z = Animal("lion", 5)
z.info()

# Exercise 2
Add the class variable number_animals = 0

Each new instance of Animal increases number_animals by 1.

In [ ]:
# code

In [ ]:
z1 = Animal("lion", 3)
z2 = Animal("tiger", 4)
print(Animal.number_animals)

# Exercise 3
Create a Bird class that inherits from Animal

Add an instance variable **type** (e.g. "Parrot")

Override the info() method to output: Name: \<name\>, Age: \<age\>, Type: \<type\>

In [ ]:
# code

In [ ]:
z3 = Bird ("bird", 50, "parrot")

# Exercise 4
Add a getter and setter to the Animal class of the _age attribute using @property and @setter.

The setter should check that the age is not negative, otherwise it will raise a ValueError.

In [ ]:
#

In [ ]:
z4 = Animal("Elephant", 10)
z4.age = -5 

# Exercise 5
Add a method \__\_del\_() to the Animal class that prints: Releasing animal \<name\>

In [ ]:
#

In [ ]:
z5 = Animal ("Cat", 30)
del z5

# Exercise 6

Add a static method that calculates the average age of the list of animals.

Add a class method that lists the number of all animals.

In [ ]:
# 